# Tool Use and Language Agents

## 一、 背景：为何语言模型需要工具？

标准的大语言模型（LMs）在文本生成方面表现出色，但它们存在固有的局限性。这些模型本质上是基于其庞大训练数据中的统计规律来生成文本的，这导致了两个核心短板：

1.  [cite\_start]**复杂的推理能力不足**：对于需要精确计算或逻辑严谨的步骤，如数学问题 $(35.223 \\times 1.6)^2 / 7 = ?$ [cite: 6, 13][cite\_start]，语言模型往往会“一本正经地胡说八道”，通过文本生成的方式去“模拟”计算，结果通常是错误的 [cite: 18, 19, 20, 21]。
2.  [cite\_start]**无法访问真实世界信息**：语言模型的知识被“冻结”在其训练数据截止的那个时间点。它们无法获取实时的、动态变化的信息，比如“今天的天气怎么样？” [cite: 7]。

[cite\_start]为了突破这些限制，研究者们提出了让语言模型使用**外部工具**（External Tools）的范式。通过调用工具，语言模型可以将其不擅长的任务（如计算、信息检索）外包给专门的程序或API，从而极大地扩展其能力边界 [cite: 8]。

这个范式催生了众多有影响力的研究，例如：

  * [cite\_start]**ToolFormer**：通过在文本中插入API调用来增强模型能力 [cite: 25]。
  * [cite\_start]**ART** 和 **ToolLLM**：探索让模型自动进行多步推理和掌握上万个真实世界API的方法 [cite: 32, 37]。
  * [cite\_start]**Gorilla** 和 **HuggingGPT**：将语言模型与海量的API或AI模型库连接起来，构建强大的任务解决系统 [cite: 54, 57]。
  * [cite\_start]**VOYAGER** 和 **TROVE**：探索让智能体在虚拟环境中自主学习，甚至自创工具来提高效率 [cite: 70, 72]。

## 二、 剖析“工具”：定义、功能与场景

### 1\. 什么是工具？

[cite\_start]我们可以从生物学的角度来理解工具。动物使用工具的定义是“利用一个外部物体来更有效地改变另一个物体的形态、位置或状态” [cite: 90, 92]。

[cite\_start]在语言模型的语境下，工具的定义类似：它是一个**外部计算机程序的函数接口** [cite: 94][cite\_start]。语言模型通过生成特定的函数调用文本（包括函数名和参数）来使用这个工具 [cite: 94]。从本质上看，所有工具都可以被视为一种**程序 (Program)**，它们可以分为两类：

  * [cite\_start]**外部 (External)**：工具是一个独立的、与模型分离的实体，如搜索引擎API、数据库查询等 [cite: 96]。
  * [cite\_start]**功能性 (Functional)**：工具是模型自身能力的一种延伸，但通过明确的接口调用，如调用一个专门用于翻译的内部模块 [cite: 97]。

### 2\. 工具的核心功能

[cite\_start]根据其作用，工具可以被归纳为三种核心功能，这与经典人工智能中对**智能体 (Agent)** 的定义不谋而合。智能体被定义为“任何能够通过**传感器 (sensors)** 感知其环境，并通过**执行器 (actuators)** 对环境采取行动的东西” [cite: 109, 110]。

  * [cite\_start]**感知 (Perception)**：通过工具从环境中收集数据和信息 [cite: 100, 101]。这相当于智能体的“传感器”。
  * [cite\_start]**行动 (Action)**：通过工具对环境施加影响，改变环境的状态 [cite: 105, 106]。这相当于智能体的“执行器”。
  * [cite\_start]**计算 (Computation)**：执行通用的计算任务，辅助模型的内部“思考”过程 [cite: 107, 108]。

### 3\. 工具使用的典型场景

[cite\_start]工具的应用场景非常广泛，可以大致分为以下几类 [cite: 135]：

| 类别 | 示例工具 | 描述 |
| :--- | :--- | :--- |
| **知识获取** | `search_engine(query)` | 访问外部知识库，克服模型知识的局限性。 |
| **计算活动** | `calculator(formula)` | 执行精确的数学运算。 |
| **与世界交互** | `get_weather(city)` | 获取实时动态信息。 |
| **处理非文本模态** | `visual_qa(query, image)` | 让模型具备处理图像、音频等多模态信息的能力。 |
| **调用其他专业模型** | `translation(text, language)` | 将特定任务（如翻译）外包给更专业的模型。 |

## 三、 核心范式：语言模型如何学会使用工具

### 1\. 基础工具使用范式

[cite\_start]最基础的工具使用是一个“生成-执行-替换”的循环 [cite: 114, 131]。

1.  [cite\_start]**生成 (Generate)**：当语言模型遇到需要工具解决的问题时，它会暂停常规的文本生成，转而生成一个特殊的、结构化的文本，即**API调用**，例如 `check_weather()` [cite: 124, 131]。
2.  [cite\_start]**执行 (Execute)**：一个外部的“调用解析器”会捕捉到这个API调用，并触发相应的工具（如天气服务器）来执行这个函数 [cite: 125, 126, 132]。
3.  [cite\_start]**替换 (Replace)**：工具执行完毕后，会返回一个结果（如 `sunny`）[cite: 130][cite\_start]。语言模型再将这个结果整合回上下文中，替换掉原来的API调用文本，并继续生成最终的答复，如“It is sunny today” [cite: 127, 128, 132]。

[cite\_start]这种模式使得模型可以在**文本生成模式**和**工具执行模式**之间灵活切换 [cite: 116, 117, 118]。

下面是一个简化的Python代码示例，模拟了这个基础的工具使用循环：


In [ ]:
import re

# 模拟的工具库
def get_weather(city: str) -> str:
    """获取指定城市的天气"""
    if city.lower() == "beijing":
        return "Sunny"
    elif city.lower() == "shanghai":
        return "Rainy"
    else:
        return "Cloudy"

def calculator(expression: str) -> str:
    """计算数学表达式"""
    try:
        # 注意：实际应用中直接用eval非常危险，这里仅为演示
        return str(eval(expression))
    except:
        return "Error: Invalid expression"

TOOL_REGISTRY = {
    "get_weather": get_weather,
    "calculator": calculator,
}

# 模拟的LLM，它会生成带有工具调用的文本
def mock_llm(prompt):
    if "weather" in prompt:
        return "The weather in Beijing is [CALL:get_weather('Beijing')] and the weather in Shanghai is [CALL:get_weather('Shanghai')]."
    elif "calculate" in prompt:
        return "The result of (5 * 8) + 10 is [CALL:calculator('(5 * 8) + 10')]."
    else:
        return "Sorry, I can't help with that."

# 工具执行引擎
def tool_executor(text_with_calls):
    call_pattern = r"\[CALL:(.*?)\]"
    
    def execute_match(match):
        call_str = match.group(1)
        # 解析函数名和参数
        func_name = call_str.split('(')[0]
        args_str = call_str[len(func_name)+1:-1]
        
        if func_name in TOOL_REGISTRY:
            # 简单解析参数，实际需要更复杂的解析器
            args = [arg.strip().strip("'\"") for arg in args_str.split(',') if arg]
            try:
                result = TOOL_REGISTRY[func_name](*args)
                return result
            except Exception as e:
                return f"Error executing tool: {e}"
        else:
            return "Error: Tool not found"

    # 使用正则表达式查找并替换所有工具调用
    final_text = re.sub(call_pattern, execute_match, text_with_calls)
    return final_text

# --- 运行示例 ---
# 1. 天气查询
prompt1 = "what is the weather like?"
llm_output1 = mock_llm(prompt1)
print(f"LLM Raw Output: {llm_output1}")
final_response1 = tool_executor(llm_output1)
print(f"Final Response: {final_response1}\n")

# 2. 计算任务
prompt2 = "calculate something"
llm_output2 = mock_llm(prompt2)
print(f"LLM Raw Output: {llm_output2}")
final_response2 = tool_executor(llm_output2)
print(f"Final Response: {final_response2}")


### 2\. 模型如何学习使用工具？

[cite\_start]让模型学会何时以及如何调用工具，主要有两种途径 [cite: 119]：

**A. [cite\_start]免训练方法：推理时提示 (Inference-time Prompting) [cite: 120]**

这是目前最流行的方法，因为它不需要重新训练模型。核心思想是在给模型的提示（Prompt）中提供所有必要信息：

1.  **任务描述**：告诉模型它的角色和目标。
2.  [cite\_start]**工具清单和文档**：明确告知模型有哪些可用的工具（API），以及每个工具的功能、参数和用法 [cite: 656, 665]。
3.  [cite\_start]**少量示例 (Few-shot Examples)**：提供一两个完整的“问题 -\> 思考过程 -\> API调用 -\> 结果 -\> 最终答案”的例子，让模型学习这个模式 [cite: 931]。
4.  [cite\_start]**思维链 (Chain-of-Thought, CoT)**：引导模型在决定调用哪个工具之前，先生成一步一步的思考过程，例如以“Let's think step by step...”开头 [cite: 614, 948]。这显著提高了模型规划和选择正确工具的能力。

**B. [cite\_start]基于训练的学习 (Learning by Training) [cite: 121]**

这种方法通过在特定的数据集上微调（Fine-tuning）语言模型，使其“内化”工具使用的能力。

  * **数据集**：需要包含大量带有正确API调用的文本样本。
  * [cite\_start]**ToolFormer的创新**：ToolFormer [cite: 25] 提出了一种巧妙的自监督学习方法。它首先让语言模型对海量文本进行扫描，在它认为调用API会有帮助的地方，主动插入API调用并执行，如果API返回的结果确实能提高原文的预测概率（即API结果很有用），那么这个新生成的 `<API调用, API结果>` 对就会被保留下来，形成微调数据。通过这种方式，模型学会了在何处、何时调用何种工具是最有益的。

### 3\. 当工具不可用时：自创工具 (TROVE)

[cite\_start]在某些场景下，可能没有现成的工具可用。TROVE [cite: 139] [cite\_start]等研究展示了语言模型甚至可以**自创工具**。其核心思想是，让一个代码语言模型（Code LM） [cite: 159] 观察大量“问题-原始解法”对，并从中归纳、抽象出可复用的高级函数（即新工具）。

[cite\_start]TROVE的工作流程如下 [cite: 181]：

1.  [cite\_start]**生成 (Generate)**：对于一个新问题，模型会生成大量不同的解决方案 [cite: 188, 160]。
2.  [cite\_start]**执行与验证 (Execute & Verify)**：执行所有生成的方案，并根据输出结果进行分组。输出结果一致的方案被认为是可能正确的 [cite: 202, 213]。
3.  [cite\_start]**抽象 (Abstract)**：在正确的方案中，寻找最简洁（操作步骤最少）的那个，并尝试将其中的逻辑抽象成一个新的函数 [cite: 217, 212]。
4.  [cite\_start]**入库 (Add to Library)**：将这个新创建的函数（工具）添加到一个工具箱中，供后续任务使用 [cite: 183, 211]。

[cite\_start]实验证明，使用TROVE自创的工具，不仅能**提升任务的准确率**，还能显著**降低解决方案的复杂性**，并使人类验证者**更快、更准确**地判断方案的正确性 [cite: 220, 226, 231, 233]。

## 四、 进化：从工具使用者到语言智能体 (LLM Agents)

[cite\_start]当语言模型不仅仅是被动地调用单个工具，而是能够进行**长期规划、记忆、并与环境持续交互**以完成复杂目标时，它就进化成了**语言智能体 (Language Agent)** [cite: 292]。

### 1\. 智能体的构成

[cite\_start]一个语言智能体系统通常由以下几个部分构成 [cite: 293]：

  * [cite\_start]**大脑 (Brain)**：通常是一个强大的语言模型（LLM），负责记忆、推理和决策 [cite: 298, 299]。
  * [cite\_start]**感知 (Perception)**：通过工具或直接的输入（文本、视觉、音频等）从环境中获取**观察 (Observations)** [cite: 302, 578]。
  * [cite\_start]**行动 (Action)**：通过调用工具或生成特定格式的指令，对环境施加影响 [cite: 307]。
  * [cite\_start]**环境 (Environment)**：智能体交互的对象，可以是网页、游戏、操作系统，甚至是真实世界 [cite: 304]。

### 2\. 智能体的训练方法

构建智能体的方法与工具学习一脉相承，但更加强调学习和适应能力。

  * [cite\_start]**免训练方法**：通过精心设计的提示工程（Prompting），让强大的LLM（如GPT-4）直接充当智能体的大脑，进行零样本或少样本的任务执行 [cite: 314, 575]。这是目前快速原型验证的主流方法。

  * [cite\_start]**监督微调 (Supervised Fine-tuning)**：通过模仿学习（Imitation Learning）来训练智能体。首先，收集大量的“专家轨迹”数据（例如，人类玩家在游戏中的操作序列 `(obs_1, action_1), ..., (obs_N, action_N)`）[cite: 1145, 1146][cite\_start]，然后用这些数据对模型进行微调，让模型学会专家的行为模式 [cite: 1108, 1147]。

      * [cite\_start]**挑战**：这种方法非常**依赖数据** [cite: 1176][cite\_start]，且模型很难从失败的轨迹中学习 [cite: 1177][cite\_start]。为了解决数据稀缺问题，研究者们提出了从互联网（如YouTube视频、Wiki文档）自动挖掘训练数据的方法 [cite: 1192, 1235]。

  * [cite\_start]**强化学习 (Reinforcement Learning, RL)**：这是训练高级智能体的终极方法 [cite: 1109, 1263]。与SFT不同，RL让智能体在与环境的**互动**中自主学习。

      * [cite\_start]**核心循环**：智能体**探索 (Explore)** 环境，收集成功和失败的**轨迹 (Trajectories)** [cite: 1304, 1330][cite\_start]。一个**奖励函数 (Reward Function)** 会根据轨迹的好坏（如任务是否成功、游戏得分等）给出奖励或惩罚 [cite: 1320, 1321, 1322, 1339][cite\_start]。智能体的目标是通过优化策略（如DPO或PPO算法）来最大化期望奖励 [cite: 1288, 1328]。
      * **优势**：RL使智能体能够从失败中学习，并发现超越人类专家的更优策略。
      * [cite\_start]**挑战**：RL面临着**环境交互成本高**、**奖励函数设计困难**和**训练不稳定**等挑战 [cite: 1346, 1347, 1349, 1351]。

下面是使用Hugging Face `transformers` 库中内置的`agent`功能来演示一个免训练智能体的代码。

In [ ]:
# 需要安装 transformers, accelerate, sentencepiece
# pip install transformers accelerate sentencepiece

import torch
from transformers import HfAgent

# 使用一个较小的、经过指令微调的模型作为智能体的大脑
# hf-internal-testing/tiny-random-starcoder 作为示例，实际可用 Starcoder, Llama 等
agent = HfAgent("https://api-inference.huggingface.co/models/bigcode/starcoder")

print("StarCoder-based agent initialized.")

# --- 示例1: 简单的数学计算 ---
# 智能体会自动生成并执行Python代码来回答问题
prompt1 = "What is the square root of 1024?"
print(f"\nUser > {prompt1}")
response1 = agent.run(prompt1)
print(f"Agent > {response1}")


# --- 示例2: 图像生成 ---
# 智能体会调用Hugging Face Hub上的图像生成模型作为工具
prompt2 = "Generate an image of a panda wearing a superhero cape."
print(f"\nUser > {prompt2}")
image = agent.run(prompt2)
print(f"Agent > Image generated. Displaying...")
# image.show() # 如果在Jupyter Notebook中，可以取消注释以显示图片

## 五、 评估：我们如何衡量智能体的能力？

评估智能体的能力是一个复杂且快速发展的领域。

### 1\. 评估基准 (Benchmarks)

[cite\_start]早期的基准主要复用现有的NLP数据集（如数学、问答）[cite: 238][cite\_start]，或者使用静态的、非交互式的环境 [cite: 721]。但这些已无法满足评估现代智能体的需求。

[cite\_start]当前，研究重心转向**交互式、长期、高难度的环境** [cite: 775, 779, 780]，例如：

  * [cite\_start]**WebShop** [cite: 742]：一个模拟在线购物网站的环境。
  * [cite\_start]**ALFWorld** [cite: 439]：一个模拟家庭环境，要求智能体完成做饭、打扫等家务任务。
  * [cite\_start]**WebArena** [cite: 798][cite\_start]：一个高度逼真和复杂的“沙盒互联网”，它自托管了购物、论坛、代码托管等多种功能完备的网站 [cite: 791, 793][cite\_start]，并要求智能体完成涉及跨网站操作的复杂任务 [cite: 781, 811]。

### 2\. 评估指标

[cite\_start]评估指标也从简单的**动作序列匹配** [cite: 729] [cite\_start]演变为**基于最终结果的评估 (Outcome-based Evaluation)** [cite: 850][cite\_start]。不再关心智能体采取的具体步骤，只关心它是否成功完成了任务目标 [cite: 852, 784][cite\_start]。例如，在WebArena中，评估器会直接检查数据库或网页内容，判断用户的订单是否正确提交、帖子是否成功发布等 [cite: 854, 874, 875]。

### 3\. 当前智能体的挑战与失败模式

[cite\_start]尽管进步显著，但即便是最强大的GPT-4模型，在WebArena这类高难度基准上的表现也远不及人类 [cite: 968]。其失败模式主要有：

  * [cite\_start]**不知如何做 (Not Knowing How)**：面对复杂任务，模型缺乏正确的规划能力，会选择错误的操作路径 [cite: 971, 973]。
  * [cite\_start]**不够精确 (Not being Accurate)**：无法处理需要严格格式的输入，如输入日期时混淆 `YYYY/MM/DD` 和 `YYYY-MM-DD` 格式 [cite: 989, 990, 995]。
  * [cite\_start]**犯低级错误 (Trivial Errors)**：出现重复输入、点击错误按钮等问题，这可能与模型的“幻觉”效应有关 [cite: 1027, 1033, 1034]。
  * [cite\_start]**对UI理解错误**：例如，在GitLab中，当被要求“将任务分配给我自己”时，模型在下拉菜单中输入了“myself”这个词，但系统中并没有这个用户，导致任务失败 [cite: 1044, 1072, 1073]。

这些失败案例凸显了当前智能体在鲁棒性、常识推理和对非结构化环境的精确理解方面仍有很长的路要走。